# Building Recommendation System

To build this model, we'll be using `surprise`, a Python [scikit](https://www.scipy.org/scikits.html) for building and analyzing recommender systems that deal with explicit rating data

You can read more about the package in the [documentation](http://surpriselib.com/) here.

In [7]:
# ! pip install scikit-surprise

In [1]:
import surprise
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from surprise.prediction_algorithms import knns, SVD, SVDpp
from surprise.similarities import cosine, msd, pearson
from surprise import Dataset, accuracy

In [2]:
# load the movielens-100k datase
data = Dataset.load_builtin('ml-100k')

### Singular Value Decomposition (SVD) Model

In [3]:
# SVD algorithm
baseline = SVD()

In [5]:
%%time

# run 5-fold cross-validation and print results.
cross_validate(baseline, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9397  0.9410  0.9372  0.9381  0.9320  0.9376  0.0031  
MAE (testset)     0.7398  0.7424  0.7382  0.7394  0.7366  0.7393  0.0019  
Fit time          4.35    4.72    4.77    5.35    4.77    4.79    0.32    
Test time         0.14    0.20    0.13    0.20    0.12    0.16    0.03    
CPU times: user 24.5 s, sys: 270 ms, total: 24.8 s
Wall time: 25.6 s


{'test_rmse': array([0.93974909, 0.94101397, 0.93723348, 0.93810943, 0.93198663]),
 'test_mae': array([0.73977153, 0.74240682, 0.73824987, 0.73943075, 0.73661823]),
 'fit_time': (4.352684020996094,
  4.717260837554932,
  4.7663726806640625,
  5.349985837936401,
  4.765328884124756),
 'test_time': (0.13851690292358398,
  0.19866228103637695,
  0.131011962890625,
  0.20126795768737793,
  0.12266206741333008)}